# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.csv

pub_date,title,venue,excerpt,citation,url_slug,paper_url
1/2/20,"Global irrigation characteristics and effects simulated by fully coupled land surface, river, and water management models in E3SM",Journal of Advances in Modeling Earth Systems,"This paper introduces a two_way coupled irrigation scheme developed within the Energy Exascale Earth System Model (E3SM). The new irrigation scheme simulates irrigation water demand and applies irrigation water in E3SM Land Model (ELM), which is coupled to a river routing model and a water management model (MOSART_WM) that simulate streamflow, reservoir operations, and irrigation water supply",,E3SM-irrigation,https://doi.org/10.1029/2020MS002069
1/4/20,Impact of climate change on water availability and its propagation through the Western U.S. power grid,Applied Energy,This paper evaluates how projected changes in water availability affect electricity generation at hydroelectric and thermal power plants and how the coincident impacts propagate loc

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [111]:
publications = pd.read_csv("add_new_pub_here.csv", sep=",", header=0, on_bad_lines='warn')
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2007-01-01,Discovery of a 1. 0 Ma Yellow River terrace an...,Progress in Natural Science,This paper reports the 4th terrace of Yellow R...,"Baotian, P., Huai, S., Chunsheng, H., Ziaofei,...",Yellow-River-Lanzhou,https://www.tandfonline.com/doi/abs/10.1080/10...
1,2012-01-01,Meander hydrodynamics initiated by river resto...,Hydrological Processes,This study quantifies how river restoration st...,"Zhou, T., & Endreny, T. (2012). Meander hydrod...",HP-CFD,https://doi.org/10.1002/hyp.8352
2,2013-01-01,Reshaping of the hyporheic zone beneath river ...,Water Resources Research,This study quantifies how river restoration st...,"Zhou, T., & Endreny, T. A. (2013). Reshaping o...",WRR-Flume-Hyporheic,https://doi.org/10.1002/WRCR.20384
3,2013-01-02,Assessing urban forest effects and values: Tor...,USDA Forest Service Report,"An analysis of trees in Toronto, Ontario","Nowak, D. J., Hoehn, R. E., III, Bodine, A. R....",Toronto-forest,https://doi.org/10.2737/nrs-rb-79
4,2014-01-01,Evaluation of real-time satellite precipitatio...,Journal of Hydrometeorology,This study evaluates the difference between TR...,"Zhou, T., Nijssen, B., Huffman, G. J., & Lette...",JHM-TRMM,https://doi.org/10.1175/JHM-D-13-0128.1
5,2014-01-02,A prototype global drought information system ...,Journal of Hydrometeorology,This paper is about a global drought monitorin...,"Nijssen, B., Shukla, S., Lin, C., Gao, H., Zho...",JHM-Drought-Monitoring,https://doi.org/10.1175/JHM-D-13-090.1
6,2016-01-01,The contribution of reservoirs to global land ...,Journal of Hydrometeorology,This study investigates human-induced water st...,"Zhou, T., Nijssen, B., Gao, H., & Lettenmaier,...",JHM-VIC-Reservoir,https://doi.org/10.1175/JHM-D-15-0002.1
7,2016-01-02,Human induced changes in the global water cycle,Terrestrial Water Cycle and Climate Change: Na...,This is a book chapter for AGU monograph about...,"Zhou, T., Haddeland, I., Nijssen, B., & Letten...",AGU-Mono,https://doi.org/10.1002/9781118971772.ch4
8,2017-01-01,Coupling a three-dimensional subsurface flow a...,Geoscientific Model Development,This paper is about coupling CLM and PFLOTRAN,"Bisht, G., Huang, M., Zhou, T., Chen, X., Dai,...",GMD-CLM-PFLOTRAN,https://doi.org/10.5194/gmd-10-4539-2017
9,2017-01-02,A New Approach to Quantify Shallow Water Hydro...,Water,This paper uses streambed temperature to predi...,"Zhou, T., Huang, M., Bao, J., Hou, Z., Arntzen...",Water-iButton,https://doi.org/10.3390/w9090703


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [112]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [113]:
import os
for row, item in publications.iterrows():
    
    year = item.pub_date[0:4]
    md_filename = str(year) + "-" + item.url_slug + ".md"
    html_filename = str(year) + "-" + item.url_slug
        
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Link to the paper](" + item.paper_url + ")\n" 
    
    md += "\n![image](../images/papers/" + str(year) + "-" + item.url_slug + ".png)\n"
    
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)
        
    item.pub_date

In [114]:
item.pub_date[0:4]

'2023'

These files are in the publications directory, one directory below where we're working from.

In [20]:
!ls ../_publications/

Invalid switch - "_publications".


In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
